<a href="https://colab.research.google.com/github/Lily999-ru/Preddiplomnaya_Praktika/blob/main/model_vkr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ИМПОРТ НЕОБХОДИМЫХ БИБЛИОТЕК

# Flask — для создания веб-приложения и обработки HTTP-запросов
from flask import Flask, render_template, request, send_file

# PyTorch — для работы с нейронной сетью и выполнения инференса модели
import torch

# torch.nn.functional — для применения функций активации и softmax
import torch.nn.functional as F

# NumPy — для работы с массивами и числовыми операциями
import numpy as np

# Pandas — для обработки CSV-файлов и табличных данных
import pandas as pd

# Transformers — для загрузки токенизатора и модели RuBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# langdetect — для автоматического определения языка входного текста
from langdetect import detect, DetectorFactory

# Исключения langdetect — для обработки ошибок определения языка
from langdetect.lang_detect_exception import LangDetectException

# os — для работы с файловой системой и путями к моделям
import os

In [ ]:
# Фиксация seed для воспроизводимости определения языка
DetectorFactory.seed = 42

# НАСТРОЙКА ПУТЕЙ И УСТРОЙСТВА

# Путь к обученной модели (ВКР версия)
MODEL_DIR = "./model"

# Выбор устройства: GPU при наличии, иначе CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ЗАГРУЗКА МОДЕЛИ И ТОКЕНИЗАТОРА
print("Загрузка токенизатора и модели RuBERT...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
model.to(device)
model.eval()
print("Модель успешно загружена.")

# СОЗДАНИЕ FLASK-ПРИЛОЖЕНИЯ
app = Flask(__name__, static_folder="static", template_folder="templates")


# ФУНКЦИЯ ПРЕДСКАЗАНИЯ С МЕХАНИЗМОМ ОТКЛОНЕНИЯ
def predict_with_rejection(text, threshold=0.45, max_length=256):

    # Проверка корректности формата входных данных
    if not isinstance(text, str) or not text.strip():
        return {"Решение": "Отклонено", "Причина": "Некорректный формат текста"}

    # Определение языка входного текста
    try:
        lang = detect(text)
    except LangDetectException:
        return {"Решение": "Отклонено", "Причина": "Не удалось определить язык текста"}

    # Отклонение, если текст не на русском языке
    if lang != "ru":
        return {
            "Решение": "Отклонено",
            "Причина": "Текст не на русском языке",
            "Язык текста": lang
        }

    # Токенизация текста
    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )

    # Перенос данных на устройство
    enc = {k: v.to(device) for k, v in enc.items()}

    # Предсказание модели без вычисления градиентов
    with torch.no_grad():
        outputs = model(**enc)
        probs = F.softmax(outputs.logits, dim=1).cpu().numpy()[0]

    # Определение наиболее вероятного класса
    max_idx = int(np.argmax(probs))
    max_prob = float(probs[max_idx])

    # Отклонение при низкой уверенности
    if max_prob < threshold:
        return {
            "Решение": "Отклонено",
            "Причина": "Низкая уверенность модели",
            "Уверенность": round(max_prob, 4)
        }

    # Успешная классификация
    return {
        "Решение": "Классифицировано",
        "label_id": max_idx,
        "label_name": model.config.id2label[max_idx],
        "Уверенность": round(max_prob, 4),
        "Язык текста": lang
    }



# НАСТРОЙКА МАРШРУТОВ
# Главная страница
@app.route("/")
def index():
    return render_template("index.html")


# Страница загрузки CSV-файла
@app.route("/csvupload", methods=["GET"])
def csvupload_page():
    return render_template("csvupload.html")


# Страница ручного ввода текста
@app.route("/textinput", methods=["GET"])
def textinput_page():
    return render_template("textinput.html")


# КЛАССИФИКАЦИЯ CSV-ФАЙЛА
@app.route("/predict_csv", methods=["POST"])
def predict_csv():

    csv_file = request.files.get("csv_file")

    # Проверка загрузки файла
    if not csv_file:
        return render_template("results.html", error="CSV файл не был загружен.")

    # Попытка чтения CSV
    try:
        df = pd.read_csv(csv_file)
    except Exception:
        return render_template("results.html", error="Ошибка чтения CSV файла.")

    # Проверка наличия колонки с текстом
    if "text" not in df.columns:
        return render_template("results.html", error="В CSV файле должна быть колонка 'text'.")

    results = []
    classified_count = 0
    rejected_count = 0

    # Классификация строк
    for _, row in df.iterrows():
        text = str(row["text"])
        title = row.get("title", "—")
        short = text[:200] + "..." if len(text) > 200 else text

        pred = predict_with_rejection(text)

        if pred["Решение"] == "Классифицировано":
            classified_count += 1
            label = pred["label_name"]
            confidence = pred["Уверенность"]
        else:
            rejected_count += 1
            label = "—"
            confidence = pred.get("Уверенность", "—")

        results.append([title, short, label, confidence])

    # Ограничение вывода первыми 5 публикациями
    visible_results = results[:5]

    # Сохранение полного результата для скачивания
    global last_results_df
    last_results_df = pd.DataFrame(
        results,
        columns=["title", "short", "label", "Уверенность"]
    )

    return render_template(
        "results.html",
        tables=visible_results,
        classified_count=classified_count,
        rejected_count=rejected_count,
        total_count=len(results)
    )



# КЛАССИФИКАЦИЯ РУЧНОГО ВВОДА
@app.route("/predict_text", methods=["POST"])
def predict_text():

    title = request.form.get("manual_title", "—")
    text = request.form.get("manual_text", "")

    # Минимальная длина текста
    if len(text.strip()) < 200:
        return render_template("results.html", error="Текст слишком короткий (минимум 200 символов).")

    short = text[:200] + "..." if len(text) > 200 else text
    pred = predict_with_rejection(text)

    results = [[title, short, pred.get("label_name", "—"), pred.get("confidence", "—")]]

    global last_results_df
    last_results_df = pd.DataFrame(
        results,
        columns=["title", "short", "label", "Уверенность"]
    )

    return render_template(
        "results.html",
        result=pred,
        title=title,
        short_text=short,
        classified_count=1 if pred["decision"] == "Классифицировано" else 0,
        rejected_count=1 if pred["decision"] != "Классифицировано" else 0,
        total_count=1
    )



# СКАЧИВАНИЕ РЕЗУЛЬТАТОВ
last_results_df = None

@app.route("/download")
def download_file():

    global last_results_df

    if last_results_df is None:
        return "Нет данных для сохранения."

    file_path = "classification_results.csv"
    last_results_df.to_csv(file_path, index=False, encoding="utf-8")

    return send_file(file_path, as_attachment=True)



# ЗАПУСК ПРИЛОЖЕНИЯ
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001, debug=True)
